<a href="https://colab.research.google.com/github/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decoding_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, style="whitegrid", palette="pastel", color_codes=True)

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_predict, LeaveOneOut
import os
import csv

In [0]:
##Download data file, set name

!wget https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
!mv 136_Data_Final.csv\?raw\=true 136_Data_Final.csv

--2019-04-12 20:50:16--  https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-12 20:50:21--  https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-12 20:50:21--  https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv
Resolving raw.githubus

In [0]:
#Import data to dataframe
##Implement Random forest stuff here...
df = pd.read_csv('136_Data_Final.csv')
df.set_index('Subject')
df.head()

,Unnamed: 0,Subject,Block,Trial,Time,O2,O1,OZ,PZ,P4,...,FC3,F3,FP2,F7,FP1,VEOU,VEOL,HEOR,HEOL,Value
0,0,136,1,1,-200,-2.6587,-5.8283,-3.6200,-8.4912,-6.0398,...,-4.2465,-6.1961,-0.7449,-4.0477,-6.5453,-252.5419,70.5294,13.6076,-66.9374,1.0
1,1,136,1,1,-196,-3.3376,-5.6611,-3.5900,-8.8003,-6.9963,...,-3.2880,-5.5053,-0.7434,-3.1528,-4.8430,-218.5595,51.3878,25.2795,-68.4394,1.0
2,2,136,1,1,-192,-3.4338,-4.6322,-2.9857,-8.3836,-7.6261,...,-2.0604,-4.9363,-0.8046,-2.6793,-2.9363,-196.0244,68.5147,18.5178,-64.9081,1.0
3,3,136,1,1,-188,-3.6411,-3.6174,-2.5697,-7.9997,-8.4763,...,-1.1900,-5.0391,-0.8101,-2.7765,-1.7893,-191.9580,51.4932,22.3986,-66.2655,1.0
4,4,136,1,1,-184,-3.9098,-3.0866,-2.5453,-8.4382,-9.8238,...,-0.9082,-5.5007,-0.6975,-3.0212,-2.2171,-177.3466,41.4210,21.8185,-66.5214,1.0


In [0]:
df_one_hot_encoded = pd.get_dummies(df)
df = df_one_hot_encoded

In [0]:
df_1 = df.groupby(np.arange(len(df))//10).mean()
df_1.head(60)

,Unnamed: 0,Subject,Block,Trial,Time,O2,O1,OZ,PZ,P4,...,FC3,F3,FP2,F7,FP1,VEOU,VEOL,HEOR,HEOL,Value
0,4.5,136.0,1.0,1.0,-182.0,-4.30124,-2.44688,-2.91630,-8.74715,-9.89857,...,-0.91213,-3.92438,-1.34789,-2.70856,-3.35473,-157.11714,29.21218,23.06317,-61.33077,1.0
1,14.5,136.0,1.0,1.0,-142.0,3.05864,4.78650,3.70634,0.17066,1.09780,...,1.91253,0.18599,2.05135,-0.50827,-1.38529,117.24946,-79.31849,19.63610,-0.98742,1.0
2,24.5,136.0,1.0,1.0,-102.0,3.08058,1.19648,2.24625,5.81754,4.14435,...,3.76300,2.50036,2.48101,3.34443,4.47601,157.05063,-67.03265,-1.36701,35.34558,1.0
3,34.5,136.0,1.0,1.0,-62.0,5.42963,-0.22368,2.65670,6.96090,10.07670,...,-4.95969,-4.57903,-3.94284,-1.11213,-0.21305,1.73968,29.56774,-17.00499,22.46689,1.0
4,44.5,136.0,1.0,1.0,-22.0,-7.26760,-3.31240,-5.69298,-4.20195,-5.42032,...,0.19630,5.81707,0.75836,0.98451,0.47708,-118.92261,87.57125,-24.32726,4.50574,1.0
5,54.5,136.0,1.0,1.0,18.0,-13.54864,-14.35524,-13.06082,-5.39458,-6.54689,...,-7.16556,-3.26763,-5.17435,-5.61662,-4.42666,-192.04056,115.67184,-40.19494,-5.02158,1.0
6,64.5,136.0,1.0,1.0,58.0,-11.59703,-3.99237,-7.01326,5.86280,-2.53551,...,3.44882,-0.04721,-8.11620,-7.30260,-7.00356,-221.83179,132.80288,-47.61037,-7.05305,1.0
7,74.5,136.0,1.0,1.0,98.0,-9.36283,-0.73690,-5.33622,-0.84654,-6.71214,...,-12.53821,-12.40654,-16.37509,-17.75678,-14.98703,-245.43473,141.39264,-48.13001,-15.11710,1.0
8,84.5,136.0,1.0,1.0,138.0,-5.90463,-2.25643,-4.24257,-0.98097,-4.81821,...,-9.42828,-10.43530,-12.78725,-14.86728,-11.50857,-251.23630,147.05446,-40.37980,-18.61476,1.0
9,94.5,136.0,1.0,1.0,178.0,-9.35645,-0.82725,-4.86715,5.53314,-1.94268,...,7.83342,-1.47516,-4.59965,-6.08353,-0.54343,-250.46442,155.19459,-34.10099,-14.28039,1.0


In [0]:
import random
random_samp_list = []
index_list = []
for i in range(1,201):
  index_list.append(i)


train_samp_list = random.sample(index_list,k=100)
  
test_list = []
test_list = [x for x in index_list if x not in train_samp_list]


#df_train = df['Trial'] == i
print(train_samp_list)
print(test_list)

[162, 158, 50, 161, 17, 11, 14, 131, 22, 149, 166, 118, 3, 164, 52, 195, 63, 38, 167, 169, 192, 101, 119, 171, 179, 172, 125, 2, 180, 68, 142, 153, 49, 165, 64, 194, 62, 99, 193, 56, 190, 196, 103, 117, 116, 184, 61, 130, 25, 71, 37, 30, 18, 200, 76, 150, 127, 181, 34, 19, 96, 72, 44, 46, 185, 113, 152, 183, 177, 59, 80, 163, 5, 20, 92, 85, 58, 98, 88, 23, 16, 83, 134, 13, 108, 1, 90, 135, 69, 32, 137, 86, 75, 128, 36, 111, 174, 8, 156, 57]
[4, 6, 7, 9, 10, 12, 15, 21, 24, 26, 27, 28, 29, 31, 33, 35, 39, 40, 41, 42, 43, 45, 47, 48, 51, 53, 54, 55, 60, 65, 66, 67, 70, 73, 74, 77, 78, 79, 81, 82, 84, 87, 89, 91, 93, 94, 95, 97, 100, 102, 104, 105, 106, 107, 109, 110, 112, 114, 115, 120, 121, 122, 123, 124, 126, 129, 132, 133, 136, 138, 139, 140, 141, 143, 144, 145, 146, 147, 148, 151, 154, 155, 157, 159, 160, 168, 170, 173, 175, 176, 178, 182, 186, 187, 188, 189, 191, 197, 198, 199]


In [0]:
rfc = RandomForestClassifier(n_estimators=150, max_features=30)
#rfc.fit(x_pd, y_pd);

In [0]:
from sklearn.model_selection import GroupShuffleSplit
n_splits = 20
cv = GroupShuffleSplit(n_splits=n_splits, test_size=0.2)
df_2 = df_1[(df_1['Value']==1) | (df_1['Value']==9)]
df_2 = df_2[df_2['Time'] < 650]
df_2 = df_2[df_2['Time'] > 450]
r_in = np.zeros(n_splits)
r_out = np.zeros(n_splits)
for i,(train, test) in enumerate(cv.split(df_2, groups=df_2['Trial'])):
  #train = df_2.index[train]
  #test = df_2.index[test]
  y = df_2['Value']
  X = df_2.drop('Value', axis=1).astype('float')
  rfc.fit(X.iloc[train].values, y.iloc[train])
  predicted = rfc.predict(X.iloc[train])
  actual = y.iloc[train]
  r_in[i] = np.corrcoef(predicted, actual)[0, 1]
  predicted = rfc.predict(X.iloc[test])
  actual = y.iloc[test]
  r_out[i] = np.corrcoef(predicted, actual)[0, 1]
  print(r_in[i], r_out[i])
print("Mean R_out = %.3g +/- %.3g" % (r_out.mean(), r_out.std()))

1.0 0.040291148201268986
1.0 0.3639472224560077
1.0 -0.0657951694959765
1.0 0.1331500571871369
1.0 -0.10040031157928
1.0 0.32748887033017876
1.0 0.31806124193615604
1.0 0.14011379937298798
1.0 0.015370070720163602
1.0 0.17702833484793162
1.0 0.11580138896327533
1.0 0.014210481288020167
1.0 -0.16481712868606588
1.0 0.12837677137225367
1.0 0.16329931618554527
1.0 0.18751092404379135
1.0 0.06579516949597693
1.0 0.1535220621572796
1.0 0.20833333333333343
1.0 0.19532924580698285
Mean R_out = 0.121 +/- 0.135


In [0]:
r

-0.34326209311093864

In [0]:
df_train = df.drop(df.index[train_samp_list])
df_test = df.drop(df.index[test_list])

    
df_x = df_train.drop(['Subject', 'Block', 'Trial'], axis=1)
df_y = df_train.drop(['Subject', 'Block', 'Trial'], axis=1)

In [0]:
y_pd = df_y['Value']
x_pd = df_x

In [0]:
y_pd_predicted = rfc.predict(x_pd)
#formula
#def report(y, y_predicted):
 # conf_mat = confusion_marix(y, y_predicted)
  #report = pd.DataFrame(index=['Value'])
  
print(y_pd_predicted)

[1. 1. 1. ... 3. 3. 3.]
